### Flask App to expose model and other computations through an API 

New model or functinlaity can be added by creating a html page and adding the function pointing to the page. Need to sun and save the word2vec model prior running the code  

In [ ]:
from gensim.models import Word2Vec
from gensim.models import Phrases

bigram=Phrases(data_words_nostops)
#sg ({0, 1}, optional) – Training algorithm: 1 for skip-gram; otherwise CBOW.

%time model = Word2Vec(bigram[data_words_nostops], size=300, window=5, min_count=5, sg=1,workers=1,iter=100)
%time model.save("/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Tuning/Payment/Pay_word2vec_skipgram.model")

Note: Change the port number (9005 to 9008) if the port is not avaliable

In [ ]:
from flask import Flask, request, render_template
from flask import render_template_string
from gensim.models import Word2Vec
from flask import Flask, jsonify, request
import csv
import nltk
from nltk.collocations import *
import collections
import pandas as pd
from termcolor import colored

app = Flask(__name__)
app.config['JSON_AS_ASCII'] = False
app.config['TEMPLATES_AUTO_RELOAD']=True

def n_concordance_tokenised(text,phrase,left_margin=15,right_margin=15):
    phraseList=phrase.split(' ')

    c = nltk.ConcordanceIndex(text.tokens, key = lambda s: s.lower())

    #Find the offset for each token in the phrase
    offsets=[c.offsets(x) for x in phraseList]
    offsets_norm=[]
    #For each token in the phraselist, find the offsets and rebase them to the start of the phrase
    for i in range(len(phraseList)):
        offsets_norm.append([x-i for x in offsets[i]])
  
    intersects=set(offsets_norm[0]).intersection(*offsets_norm[1:])

    concordance_txt = ([text.tokens[list (map(lambda x: x-left_margin if (x-left_margin)>0 else 0,[offset]))[0]:offset+len(phraseList)+right_margin]
                    for offset in intersects])

    outputs=[''.join([x+' ' for x in con_sub]) for con_sub in concordance_txt]
    return outputs

@app.route('/')
def home():
    return render_template('home.html')


@app.route('/', methods=['POST'])
def word2vec():
    model = Word2Vec.load("/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Tuning/PIU/PIU_word2vec_skipgram.model")
    w = request.form['text']
    tn=request.form['topN']
    #print tn
    top_n=int(tn)
    #print top_n
    #wordcloud = WordCloud(width=1200, height=1000, margin=0,background_color="white",min_font_size=10).generate_from_frequencies(model.wv.most_similar(w, topn=200)
    w2v=pd.DataFrame(model.wv.most_similar(w, topn=top_n))
    w2v.columns=['Word','Weigtage']
    w2v_result=w2v.to_dict('records')
    w2v_columnNames = w2v.columns.values
    
    return render_template('table.html',records=w2v_result, colnames=w2v_columnNames)

@app.route('/WordAlgebra/')
def WordAlgebra():
    return render_template('WordAlgebra.html')

@app.route('/WordAlgebra/', methods=['POST'])
def word2vec_2():
    model1 = Word2Vec.load("/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Tuning/PIU/PIU_word2vec_skipgram.model")
    add_elements = [y for y in (x.strip() for x in request.form['add'].split(',')) if y]
    a=[x.encode('utf-8') for x in add_elements]
    sub_elements = [y for y in (x.strip() for x in request.form['sub'].split(',')) if y]
    s=[x.encode('utf-8') for x in sub_elements]
    t_wa=request.form['top_wa']
    #print tn
    #t=int(t_wa)
    df_wa=pd.DataFrame(model1.wv.most_similar(positive=a, negative=s, topn=int(t_wa)))
    df_wa.columns=['Word','Weigtage']
    df_wa_result=df_wa.to_dict('records')
    df_wa_columnNames = df_wa.columns.values
    
    return render_template('table.html',records=df_wa_result, colnames=df_wa_columnNames)
    
    #return jsonify(model1.most_similar(positive=a, negative=s, int(t_wa)))

@app.route('/WordPrefix/')
def WordPrefix():
    return render_template('WordPrefix.html')

@app.route('/WordPrefix/', methods=['POST'])
def get_prefix_word():
    # load data
    with open('/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Tuning/PIU/data_words_trigrams_upd.csv','rU') as f1:
        data_words=list( csv.reader(f1) )
    #flatten the lists
    all_lemma_words = [y for x in data_words for y in x]
    bgm    = nltk.collocations.BigramAssocMeasures()
    finder = nltk.collocations.BigramCollocationFinder.from_words(all_lemma_words)
    scored = finder.score_ngrams( bgm.likelihood_ratio  )
    prefix_keys = collections.defaultdict(list)
    
    for key, scores in scored:
        prefix_keys[key[0]].append((key[1], scores))
    
    for key in prefix_keys:
        prefix_keys[key].sort(key = lambda x: -x[1])
        
    p_word=request.form['p_word']
    p_length=request.form['p_length']
    
    l=int(p_length)
    pw=pd.DataFrame(prefix_keys[p_word][:l])
    pw.columns=['Word','Weigtage']
    temp=pw.to_dict('records')
    columnNames = pw.columns.values
    
    
    #return jsonify(prefix_keys[p_word][:l])
    return render_template('table.html',records=temp, colnames=columnNames)
@app.route('/concordance/')
def concordance():
    return render_template('concordance.html')

@app.route('/concordance/', methods=['POST'])
def get_word_conc():
    # load data
    with open('/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Tuning/PIU/data_words_trigrams_upd.csv','rU') as f1:
        data_words=list( csv.reader(f1) )
    
#flatten the lists
    all_lemma_words = [y for x in data_words for y in x]
    
    c_word=request.form['c_word']
    
    p_text = nltk.Text(all_lemma_words)
    concordance_txt=n_concordance_tokenised(p_text,c_word)
    print c_word
    before=[]
    after=[]
    center=[]
    for i in range(len(concordance_txt)):
        before.append(concordance_txt[i][:15])
        center.append(concordance_txt[0][15:17])
        after.append(concordance_txt[i][17:])
    
    conc_df=pd.DataFrame(list(zip(before, center, after)),columns=['Before_Words','Searched Phrase/Word', 'After_Words']) 
    recz=conc_df.to_dict('records')
    c_columnNames = conc_df.columns.values
    return jsonify(n_concordance_tokenised(p_text,c_word))
    #return render_template('conc_op.html',records=recz, colnames=c_columnNames)
 
@app.route('/OpinionMining/')
def OpinionMining():
    return render_template('sentiment.html')

@app.route('/OpinionMining/', methods=['POST'])
def sentiment():
    sdf=pd.read_csv("/data1/call_miner/Vinyas_Call_miner/call_miner/Lavanya/Sentiment_Model/sent_score.csv")
    sword=request.form['s_word']
    call_vol=len(sdf[[sword in x for x in sdf['Data_words']]])
    sub_df=sdf[[sword in x for x in sdf['Data_words']]]
    pos=len(sub_df.loc[sub_df['compound'] >=0.05])
    neg=len(sub_df.loc[sub_df['compound']<=-0.05])
    neu=len(sub_df[(sub_df['compound']> -0.05) & (sub_df['compound']<0.05)])
    score=[call_vol,pos,neg,neu]
    df3=pd.DataFrame(score).T
    df3.rename(columns={0:'Total CallHit Count',1:'PositiveCount',2:'NegativeCount',3:'NeutralCount'},inplace=True)
    
    stemp=df3.to_dict('records')
    s_columnNames = df3.columns.values
    
    
    #return jsonify(prefix_keys[p_word][:l])
    return render_template('table.html',records=stemp, colnames=s_columnNames)
    
    #return jsonify(model1.most_similar(positive=a, negative=s, int(t_wa)))

if __name__ == '__main__':
    app.run('10.70.85.50',port=9007,debug=False)

 * Running on http://10.70.85.50:9007/ (Press CTRL+C to quit)
10.72.147.207 - - [05/Jan/2019 13:17:22] "GET /OpinionMining/ HTTP/1.1" 200 -
10.72.147.207 - - [05/Jan/2019 13:17:22] "GET /OpinionMining/ HTTP/1.1" 200 -
10.72.147.207 - - [05/Jan/2019 13:17:26] "GET /OpinionMining/ HTTP/1.1" 200 -
10.72.147.207 - - [05/Jan/2019 13:17:56] "POST /OpinionMining/ HTTP/1.1" 200 -
10.72.147.207 - - [05/Jan/2019 13:18:02] "GET /OpinionMining/ HTTP/1.1" 200 -
